# Load GeoQuery Dataset

### GeoQuery dataset property:
1. logic forms are different (probably could rewrite ourselves)
2. We have could parse to get the query
3. Cross-schema questions (e.g. "what,are,the,populations,of,all,the,major,cities,in,montana","which,capitals,are,in,the,states,that,border,texas")
--- Solution: create two logical predicates: 'border' and 'in': states border states, cities in states; type 'array' with new predicate


### Following is a description of the geobase predicates:

state(name, abbreviation, capital, population, area, state_number, city1, city2, city3, city4)

city(state, state_abbreviation, name, population)

river(name, length, [states through which it flows])

border(state, state_abbreviation, [states that border it])

highlow(state, state_abbreviation, highest_point, highest_elevation, lowest_point, lowest_elevation)

mountain(state, state_abbreviation, name, height)

road(number, [states it passes through])

lake(name, area, [states it is in])


### We can change to the format we want:

state, abbreviation, capital, population, area, state_number, major_cities, states_border, highest_point, highest_elevation, lowest_point, lowest_elevation, rivers, mountains, lakes

city, state, abbreviation, population

mountain, state, abbreviation, height

river, length, states_through

(road, states_through) (No questions)

lake, area, states_in


### How to do that? Transform a graph-structure knowledge base to table-like is easy.
1. Read in 'base' file to get all the values
2. structured graph to table
3. analyze logical forms and rewrite
4. data augmentation and train

In [10]:
import numpy as np
import re

def strIsNum(s):
    '''verify if the word represent a numerical value
    ''' 
    if not isinstance(s, basestring):
        return 0
    ones = {"zero", "one", "two", "three", "four", "five", "six", "seven", "eight", "nine"}
    tens = {"twenty", "thirty", "forty", "fifty", "sixty", "seventy", "eighty", "ninety"}
    teens = {"ten", "eleven", "twelve", "thirteen", "fourteen", "fifteen", "sixteen", \
            "seventeen", "eighteen", "nineteen"}
    levels = {"hundred", "thousand", "million", "billion", "trillion"}
    if s in ones:
        return True 
    if s in tens:
        return True 
    if s in teens:
        return True 
    if s in levels:
        return True 
    if s.isdigit():
        return True #return int(s)
    # for float value
    try:
        x = float(s)
        return True
    except ValueError:
        return False
    return False


In [15]:
#f = open('./')
s = "state('alabama',['al','montgomery',3894.0e+3,51.7e+3,22,'birmingham'],'mobile','montgomery','huntsville')."
m = re.findall(r'\[([^()]+)\]', s)
#m = re.findall(r'\(([^()]+)\)', s)

entity = s[:s.find("(")]

print entity
print m
strIsNum('3894.0e+3')

state
["'al','montgomery',3894.0e+3,51.7e+3,22,'birmingham'"]


True

In [21]:
with open('./GeoQuery/geoqueries880','r') as f_query:
    for line in f_query:
        #print line
        extract = re.findall(r'\[([^()]+)\]', line)
        #print extract
        tokens = extract[0].split(',')
        print ' '.join(tokens[:-1])

give me the cities in virginia
what are the high points of states surrounding mississippi
name the rivers in arkansas
name all the rivers in colorado
can you tell me the capital of texas
could you tell me what is the highest point in the state of oregon
count the states which have elevations lower than what alabama has
give me all the states of usa
give me the cities in texas
give me the cities in usa
give me the cities in virginia
give me the cities which are in texas
give me the lakes in california
give me the largest state
give me the longest river that passes through the us
give me the number of rivers in california
give me the states that border utah
how big is alaska
how big is massachusetts
how big is new mexico
how big is north dakota
how big is texas
how big is the city of new york
how high are the highest points of all the states
how high is guadalupe peak
how high is mount mckinley
how high is the highest point in america
how high is the highest point in montana
how high is 

In [ ]:
with open('./GeoQuery/geobase','r') as f_base:
    idx = 0
    for line in f_base:
        idx += 1
        if idx < 23:
            continue
        elif idx < 74:
            # read state schema
        elif idx < 460:
            # read city
        elif idx < 506:
            # read river
        elif idx < 557:
            # read border:
        elif idx < 608:
            # read highlow
        elif idx < 658:
            # read mountain
        elif idx < 698:
            # read road
        elif idx < 720:
            # read lake
        